In [181]:
# whether to log each feature and sequence status
verbose = True

In [82]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import json
import cv2
from keras.preprocessing.image import img_to_array
from sklearn.utils import shuffle

In [2]:
# setup paths
pwd = os.getcwd().replace("deepvideoclassification","")
path_cache = pwd + 'cache/'
path_data = pwd + 'data/'

In [3]:
# setup logging
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])
logger = logging.getLogger()

In [ ]:
# read vid folders
def get_video_paths():
    """
    Return list of video paths 

    Videos should be in /data/video_1/, /data/video_2/ style folders 
    with sequentially numbered frame images e.g. /data/video_1/frame00001.jpg

    There should be at least 3 videos, 1 for each of train/test/valid splits
    Split assignment is given in /data/labels.csv (also to be provided by user)

    Functionality to use different parts of a video as train/valid/test 
    not currently implemented.
    """
    path_videos = []
    for filename in os.listdir(path_data):
        if os.path.isdir(os.path.join(path_data, filename)):
            path_videos.append(filename)

    path_videos = [path_data + v + '/' for v in path_videos]

    # make sure that there is video data in /data/ and give instructions if not done correctly
    assert len(path_videos)>0, "There need to be at least 3 video folders (at least 1 for each of train, valid, \
    and test splits) in /data/ - each video should be its own folder of frame images with ascending time-ordered \
    filenames e.g. /data/vid1/frame00001.jpg ... videos assignment to train/valid/test split should be given in \
    /data/labels.csv ... cross-validation or train/valid/test splits within a single long video not currently implemented"

    return path_videos

In [184]:
def resize_frames(target_size):
    """
    Resize the frames of all videos and save them to /cache/ 
    to make model fitting faster .

    We resize once upfront rather than each we use a pretrained model or architecture.

    Our models require inputs resized to:
    * 224 x 224 VGG16, ResNet50, DenseNet, MobileNet
    * 299 x 299 XCeption, InceptionV3, InceptionResNetV2
    * 112 x 112 3D CNN 
    """

    if not os.path.exists(path_cache + 'frames/' + str(target_size[0]) + "_" + str(target_size[1]) + '/'):

        # read vid paths
        path_videos = get_video_paths()

        # loop over all vids and resize frames, saving to new folder in /cache/frames/
        for c,path_video in enumerate(path_videos):

            logger.info("resizing vid {}/{} to {}x{}".format(c+1,len(path_videos),target_size[0], target_size[1]))

            # get vid name from path
            video_name = path_video.split("/")[-2]

            # create directory for resized frames
            # e.g. path_vid_resized = /cache/frames/224_224/s23-4847/
            path_vid_resized = path_cache + 'frames/'
            path_vid_resized += str(target_size[0]) + "_" + str(target_size[1]) + '/' 
            path_vid_resized += video_name + '/'
            if not os.path.exists(path_vid_resized):
                os.makedirs(path_vid_resized)

            # load frame paths for vid
            path_frames = os.listdir(path_vid)
            path_frames = [path_video + f for f in path_frames if f != '.DS_Store']
            path_frames.sort()

            # load frames
            frames = []
            for path_frame in path_frames:

                # open image and resize
                filename = path_frame.split("/").pop()
                img_frame = Image.open(path_frame)
                img_frame = img_frame.resize(target_size)
                # img_frame.save(path_vid_resized + filename, "JPEG", quality = 100)

                # convert to array and append to list
                img_frame = np.array(img_frame)
                frames.append(img_frame)

            # save array of resized frames
            np.save("/mnt/seals/cache/frames/" + str(target_size[0]) + "_" + str(target_size[1]) + "/" + video_name, np.array(frames))

In [185]:
def get_labels():
    # read labels - should be CSV with columns "video","frame","label","split"
    # e.g. "s1-218", "s1-218-00001.jpeg", "noseal", "train"
    labels = None
    try:
        labels = pd.read_csv(path_data + 'labels.csv', usecols=['video','frame','label','split'])
    except ValueError as e:
        raise Exception("Labels file must contain columns ['video','frame','label','split'] - if you only have ['video','frame','label'], use the helper function add_splits_to_labels_file to add train/valid/test splits to your labels file")
    except FileNotFoundError as e:
        raise Exception("No labels found - please save labels file to /data/labels.csv") from e

    return labels.sort_values(["video","frame"])

In [186]:
def create_video_label_arrays():
    """
    Create numpy array with labels for each vid and a label_map.json file
    in /cache/labels/
    """

    # create folder for labels
    if not os.path.exists(path_cache + 'labels/'):
        os.makedirs(path_cache + 'labels/')

    # load labels
    labels = get_labels()

    # build label_map
    label_dummies = pd.get_dummies(labels, columns = ['label'])

    # get label columns list and build label map dict
    label_columns = []
    label_map = {}
    label_map_idx = 0
    for i, col in enumerate(label_dummies.columns):
        if col[:6] == 'label_':
            label_columns.append(col)
            label_map[label_map_idx] = col
            label_map_idx+=1

    # save label map to json
    with open(path_cache + 'labels/label_map.json', 'w') as fp:
        json.dump(label_map, fp)

    # get video paths
    path_videos = get_video_paths()

    # save numpy array of labels for each vid
    for path_video in path_videos:

        # get vid name from path
        video_name = path_video.split("/")[-2]

        vid_labels = np.array(label_dummies[label_dummies['video'] == video_name][label_columns])

        # save labels array for this vid
        np.save("/mnt/seals/cache/labels/" + video_name, np.array(vid_labels))

In [187]:
def load_label_map():
    """
    Returns label map - read from disk
    """

    # load label map from disk
    label_map = None
    try:
        if os.path.exists(path_cache + 'labels/label_map.json'):
            with open(path_cache + 'labels/label_map.json', 'r') as fp:
                label_map = json.load(fp)
        else:
            # build labels and label map
            create_video_label_arrays()
            if os.path.exists(path_cache + 'labels/label_map.json'):
                with open(path_cache + 'labels/label_map.json', 'r') as fp:
                    label_map = json.load(fp)
    except Exception as e:
        logger.error ('label map not found - make sure /data/labels.csv exists and data cache has been built')

    return label_map

In [188]:
def load_pretrained_model(pretrained_model_name, pooling):
    """ Load pretrained model with given pooling applied
    
    Args:
        pretrained_model: name of pretrained model ["Xception", "VGG16", "ResNet50", "InceptionV3", "InceptionResNetV2", "MobileNetV2"]
        pooling: pooling strategy for final pretrained model layer ["max","avg"]
    
    Returns:
        Pretrained model object (excluding dense softmax 1000 ImageNet classes layer)
    """
    
    # initialize output
    model = None
    
    ###########################
    ### import pretrained model
    ###########################
    if pretrained_model_name == "xception":   
        from keras.applications.xception import Xception
        model = Xception(include_top=False, weights='imagenet', pooling=pooling)
    elif pretrained_model_name == "vgg16":   
        from keras.applications.vgg16 import VGG16
        model = VGG16(include_top=False, weights='imagenet', pooling=pooling)
    elif pretrained_model_name == "resnet50":   
        from keras.applications.resnet50 import ResNet50
        model = ResNet50(include_top=False, weights='imagenet', pooling=pooling)
    elif pretrained_model_name == "inception_v3":   
        from keras.applications.inception_v3 import InceptionV3
        model = InceptionV3(include_top=False, weights='imagenet', pooling=pooling)
    elif pretrained_model_name == "inception_resnet_v2":   
        from keras.applications.inception_resnet_v2 import InceptionResNetV2
        model = InceptionResNetV2(include_top=False, weights='imagenet', pooling=pooling)
    elif pretrained_model_name == "mobilenetv2_1.00_224":   
        from keras.applications.mobilenet_v2 import MobileNetV2
        model = MobileNetV2(include_top=False, weights='imagenet', pooling=pooling)
    else:
        raise NameError('Invalid pretrained model name - must be one of ["Xception", "VGG16", "ResNet50", "InceptionV3", "InceptionResNetV2", "MobileNetV2"]')
    
    return model

In [189]:
def load_pretrained_model_preprocessor(pretrained_model_name):
    """
    Return preprocessing function for a given pretrained model
    """

    preprocess_input = None
    
    if pretrained_model_name == "xception":   
        from keras.applications.xception import preprocess_input
    elif pretrained_model_name == "vgg16":   
        from keras.applications.vgg16 import preprocess_input
    elif pretrained_model_name == "resnet50":   
        from keras.applications.resnet50 import preprocess_input
    elif pretrained_model_name == "inception_v3":   
        from keras.applications.inception_v3 import preprocess_input
    elif pretrained_model_name == "inception_resnet_v2":   
        from keras.applications.inception_resnet_v2 import preprocess_input
    elif pretrained_model_name == "mobilenetv2_1.00_224":   
        from keras.applications.mobilenet_v2 import preprocess_input
    else:
        raise NameError('Invalid pretrained model name - must be one of ["Xception", "VGG16", "ResNet50", "InceptionV3", "InceptionResNetV2", "MobileNetV2"]')
        
    return preprocess_input

In [190]:
# define pretrained model shapes
pretrained_model_shapes = {}
#
pretrained_model_shapes['vgg16'] = (224,224)
pretrained_model_shapes['resnet50'] = (224,224)
pretrained_model_shapes['mobilenetv2_1.00_224'] = (224,224)
#
pretrained_model_shapes['xception'] = (299,299)
pretrained_model_shapes['inception_v3'] = (299,299)
pretrained_model_shapes['inception_resnet_v2'] = (299,299)

In [161]:
def precompute_CNN_features(pretrained_model_name, pooling):
    """ Save pretrained features array computed over all frames of each video using given pretrained model and pooling method
    
    Args:
        pretrained_model_name: pretrained model object loaded using `load_pretrained_model`
        pooling: pooling method used with pretrained model
    
    Returns:
        None. Saves pretrained frames to `/cache/features/*pretrained_model_name*/*pooling*/*video_name*.npy`
    """
    
    # load pretrained model
    pretrained_model = load_pretrained_model(pretrained_model_name, pooling)
    
    # load preprocessing function
    preprocess_input = load_pretrained_model_preprocessor(pretrained_model_name)
    
    # lookup pretrained model input shape
    model_input_shape = pretrained_model_shapes[pretrained_model_name]
    
    # setup path to save featuers
    path_features = path_cache + 'features/' + pretrained_model_name + "/" + pooling + '/'
    if not os.path.exists(path_features):
        os.makedirs(path_features)
        
    path_videos = get_video_paths()
    
    for c, path_video in enumerate(path_videos):
        
        if verbose:
            logging.info("Computing pretrained model features for video {}/{} using pretrained model: {}, pooling: {}".format(c+1,len(path_videos),pretrained_model_name, pooling))
        
        # get video name from video path
        video_name = path_video.split("/")[-2]
    
        # build output path
        path_output = path_features + video_name

        try:
            if not os.path.exists(path_output + '.npy'):

                path_frames = path_data + video_name + "/"

                # initialize features list
                features = []

                frame_paths = os.listdir(path_frames)
                frame_paths = [path_frames + f for f in frame_paths if f != '.DS_Store']

                # sort paths in sequence (they were created with incrementing filenames through time)
                frame_paths.sort()

                # load each frame in vid and get features
                for j, frame_path in enumerate(frame_paths):

                    # load image & preprocess
                    image = cv2.imread(frame_path, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(image, model_input_shape, interpolation=cv2.INTER_AREA)
                    img = img_to_array(img)
                    img = np.expand_dims(img, axis=0)
                    img = preprocess_input(img)

                    # get features from pretrained model
                    feature = pretrained_model.predict(img).ravel()
                    features.append(feature)

                # convert to arrays
                features = np.array(features)

                np.save(path_output, features)
            else:
                if verbose:
                    logger.info("Features already cached: {}".format(path_output))

        except Exception as e:
            logging.error("Error precomputing features {} / {},{}".format(video_namepretrained_model_name, pooling))
            logging.fatal(e, exc_info=True)

    return True

In [218]:
class Data(object):
    
    def __init__(self, sequence_length, target_size, return_CNN_features = False, 
                 pretrained_model_name = None, pooling = None, augmentation = False, oversampling = False):
        """
        Data object constructor
        
        :sequence_length: number of frames in sequence to be returned by Data object
        :target_size: size that frames are resized to (different models / architectures accept different input sizes)

        :pretrained_model_name: name of pretrained model (or None if not using pretrained model e.g. for 3D-CNN)
        :pooling: name of pooling variant (or None if not using pretrained model e.g. for 3D-CNN)
        :return_features: if True then return features (or sequences of feature) from pretrained model, if False then return frames (or sequences of frames)        
        :augmentation: whether to apply data augmentation (horizontal flips)
        :oversampling: whether to apply oversampling to create class balance
        """
    
        # required params
        self.sequence_length = sequence_length
        self.target_size = target_size
        
        # optional params
        self.pretrained_model_name = pretrained_model_name
        self.pooling = pooling
        self.return_CNN_features = return_CNN_features
        self.augmentation = augmentation
        self.oversampling = oversampling
        
        # model data
        self.x_train = []
        self.y_train = []
        #
        self.x_valid = []
        self.y_valid = []
        # 
        self.x_test = []
        self.y_test = []
        
        
        ################
        ### Prepare data
        ################
        
        # get video paths
        self.path_videos = get_video_paths()
        
        # create label array for each video and load label map
        create_video_label_arrays()
        self.label_map = load_label_map()
        
        # get labels
        self.labels = get_labels()
        
        # create dict mapping video to train/valid/test split assignment
        video_splits = d.labels[['video','split']].drop_duplicates()
        video_splits.set_index("video", inplace=True)
        video_splits = video_splits.to_dict()['split']
        self.video_splits = video_splits
        
        # resize frames to correct size if not already resized
        resize_frames(target_size)

        # pre compute CNN features if not already computed
        if return_CNN_features and pretrained_model_name is not None:
            precompute_CNN_features(self.pretrained_model_name, self.pooling)
        
        # load features/frames from all videos and concat into big array for each of train, valid and test
        if self.sequence_length > 1:
            
            ###################
            ### frame sequences
            ###################
            
            if return_CNN_features:
                
                #####################
                ### feature sequences
                #####################
                
                path_features = path_cache + 'features/' + pretrained_model_name + "/" + pooling + '/'
                path_labels = path_cache + 'labels/'
                
                # read vid paths
                path_videos = get_video_paths()

                # loop over all vids and resize frames, saving to new folder in /cache/frames/
                for c, path_video in enumerate(path_videos):

                    # get vid name from path
                    video_name = path_video.split("/")[-2]
                    
                    
                    ### create sequence: features
                    # load precomputed features
                    features = np.load(path_features + video_name + '.npy')
                    # build sequences
                    x = []
                    for i in range(sequence_length, len(features) + 1):
                        x.append(features[i-sequence_length:i])
                    x = np.array(x)
                    
                    
                    ### create sequence: features
                    # load precomputed features
                    labels = np.load(path_labels + video_name + '.npy')     

                    # temp lists to store sequences
                    y = []
                    for i in range(sequence_length, len(labels) + 1):
                        y.append(labels[i-1])
                    y = (np.array(y))

                    
                    ### build output
                    if self.video_splits[video_name] == "train":
                        self.x_train.append(x)
                        self.y_train.append(y)
                    if self.video_splits[video_name] == "valid":
                        self.x_valid.append(x)
                        self.y_valid.append(y)
                    if self.video_splits[video_name] == "test":
                        self.x_test.append(x)
                        self.y_test.append(y)
                        
            else:

                ###################
                ### frame sequences
                ###################
                
                path_vid_resized = path_cache + 'frames/'
                path_vid_resized += str(self.target_size[0]) + "_" + str(self.target_size[1]) + '/' 
                
                path_labels = path_cache + 'labels/'
                
                # read vid paths
                path_videos = get_video_paths()

                # loop over all vids and resize frames, saving to new folder in /cache/frames/
                for c, path_video in enumerate(path_videos):

                    # get vid name from path
                    video_name = path_video.split("/")[-2]
                    
                    ### create sequence: features
                    # load precomputed features
                    frames = np.load(path_vid_resized  + video_name + '.npy')
                    # build sequences
                    x = []
                    for i in range(sequence_length, len(frames) + 1):
                        x.append(frames[i-sequence_length:i])
                    x = np.array(x)
                    
                    
                    ### create sequence: features
                    # load precomputed features
                    labels = np.load(path_labels + video_name + '.npy')     

                    # temp lists to store sequences
                    y = []
                    for i in range(sequence_length, len(labels) + 1):
                        y.append(labels[i-1])
                    y = (np.array(y))

                    ### build output
                    if self.video_splits[video_name] == "train":
                        self.x_train.append(x)
                        self.y_train.append(y)
                    if self.video_splits[video_name] == "valid":
                        self.x_valid.append(x)
                        self.y_valid.append(y)
                    if self.video_splits[video_name] == "test":
                        self.x_test.append(x)
                        self.y_test.append(y)
                
                
        else:
            ###############
            ### no sequence
            ###############
            if return_CNN_features:
                
                ###################
                ### feature vectors
                ###################
                
                path_features = path_cache + 'features/' + pretrained_model_name + "/" + pooling + '/'
                path_labels = path_cache + 'labels/'
                
                # read vid paths
                path_videos = get_video_paths()

                # loop over all vids and resize frames, saving to new folder in /cache/frames/
                for c, path_video in enumerate(path_videos):

                    # get vid name from path
                    video_name = path_video.split("/")[-2]
                    
                    ### load precomputed features
                    x = np.load(path_features + video_name + '.npy')
                    y = np.load(path_labels + video_name + '.npy')
                    
                    ### build output
                    if self.video_splits[video_name] == "train":
                        self.x_train.append(x)
                        self.y_train.append(y)
                    if self.video_splits[video_name] == "valid":
                        self.x_valid.append(x)
                        self.y_valid.append(y)
                    if self.video_splits[video_name] == "test":
                        self.x_test.append(x)
                        self.y_test.append(y)
            else:
                
                #################
                ### single frames
                #################
                
                path_vid_resized = path_cache + 'frames/'
                path_vid_resized += str(self.target_size[0]) + "_" + str(self.target_size[1]) + '/' 
                
                path_labels = path_cache + 'labels/'
                
                # read vid paths
                path_videos = get_video_paths()

                # loop over all vids and resize frames, saving to new folder in /cache/frames/
                for c, path_video in enumerate(path_videos):

                    # get vid name from path
                    video_name = path_video.split("/")[-2]
                    
                    # load precomputed numpy arrays for frames and labels
                    x = np.load(path_vid_resized  + video_name + '.npy')
                    y = np.load(path_labels + video_name + '.npy')
                
                    ### build output
                    if self.video_splits[video_name] == "train":
                        self.x_train.append(x)
                        self.y_train.append(y)
                    if self.video_splits[video_name] == "valid":
                        self.x_valid.append(x)
                        self.y_valid.append(y)
                    if self.video_splits[video_name] == "test":
                        self.x_test.append(x)
                        self.y_test.append(y)
            
        ########################
        ### reshape list outputs
        ########################
        ## e.g. (9846, 224, 224, 3) for frames [return_CNN_features=True]
        ## or  (9846, 512) for features [return_CNN_features=False]
        self.x_train = np.concatenate(self.x_train, axis=0)
        self.y_train = np.concatenate(self.y_train, axis=0)
        self.x_valid = np.concatenate(self.x_valid, axis=0)
        self.y_valid = np.concatenate(self.y_valid, axis=0)
        self.x_test = np.concatenate(self.x_test, axis=0)
        self.y_test = np.concatenate(self.y_test, axis=0)
            

        ###########
        ### shuffle
        ###########
        self.x_train, self.y_train = shuffle(self.x_train, self.y_train)
        self.x_valid, self.y_valid = shuffle(self.x_valid, self.y_valid)

# TODO

> refactor pretrained model into models

> make sure don't recompute sequences if inputs don't change